This notebook allows you to visualize exploration and exploitation statistics for the Exploration in Malmo Continuous Mountain Cart EMC^2 experiment. make sure your data is in the run_saves/ folder.

In [ ]:
from utils.gep_utils import Bounds
import numpy as np
import os
import pylab
import seaborn
import config
import scipy.stats as sp
import pickle
import matplotlib.pyplot as plt
import imageio
from scipy.spatial import distance
from collections import OrderedDict

In [ ]:
# Step 1 load data
nb_runs = 9
model_full_names = {'rgb':"Flat Random Goal Babbling", 'rmb':"Random Model Babbling",
                    'amb':"Active Model Babbling", 'sgs': "Single Goal Space",
                    'random':"RANDOM"}
#model_full_names = ["Flat Random Goal Babbling", "Random Model Babbling", "Active Model Babbling"]
#model_full_names = ["Flat Random Goal Babbling", "Random Model Babbling", "Active Model Babbling"]
#model_filenames = ["emmcnewcodenewint_f_rgb","emmcnewcodenewint_rmb","emmcnewcodenewint_amb"]
titles = {'rgb':'F-RGB', 'rmb':'RMB', 'amb':'AMB',
          'random':'RANDOM', 'sgs':'SGS'}
model_filenames = ["emmc3d_amb"]
#model_filenames = ["emmcseqnewenv_f_rgb","emmcseqnewenv_rmb","emmcseqnewenv_amb"]
#model_filenames = ["emmc10dfinal_rmb","emmc10dfinal_amb"]

#model_filenames = ["random","dist_fmb","dist_rmb","dist_amb"]
nb_models = len(model_filenames)
model_colors = {'random': u'#8c564b', 'sgs': u'#d62728', 'amb': u'#2ca02c', 'rmb': u'#ff7f0e', 'rgb': u'#1f77b4'}
#colors = [u'#8c564b', u'#d62728',u'#ff7f0e', u'#1f77b4', u'#2ca02c']

    

saves = OrderedDict()
gep_saves = {}
stat = OrderedDict()
model_ids = []
for k in model_filenames:
    k = k.split('_')[-1]
    model_ids.append(k)
    saves[k] = []
    gep_saves[k] = []
    stat[k] = {}


for filename in model_filenames:
    print(filename)
    for j in range(nb_runs):
        full_filename = "run_saves/"+filename+"_"+str(j)+"_bk.pickle"
        #full_save_name = "run_saves/"+filename+"_"+str(j)+"_save.pickle"
        if os.path.isfile(full_filename):
            data = pickle.load(open(full_filename,"rb"))
            #gep_data = pickle.load(open(full_save_name,"rb"))
            saves[filename.split('_')[-1]].append(data)
            print(filename.split('_')[-1])
            #gep_saves[filename].append(gep_data[0])
            nb_episodes = len(data['end_agent_x'])
            print(nb_episodes)
        else:
            print("unknown file: {}".format(full_filename))

In [ ]:
def plot_agent_pos_exploration(ax, Xs, Zs,cart_Xs, title):
    b = Bounds()
    b.add('agent_x',[288.3,294.7])
    b.add('agent_z',[433.3,443.7])

    # add arena boundaries 
    points = np.array([[288.3, 433.3],
                       [294.7, 433.3],
                       
                       [294.7,443.3],
                       [294.7,443.7],
                       [288.3,443.7],
                       [288.3,443.3]])

    #upper floor limit
    #ax.plot([293.7, 292.3], [440.7, 440.7], color='k', linestyle=':', linewidth=2)
    #ax.plot([290.7, 289.3], [440.7, 440.7], color='k', linestyle=':', linewidth=2)

    #cart swinged up limit
    #ax.plot([286.8, 286.8], [443, 444], color='g', linestyle='--', linewidth=2)
    #ax.plot([296.2, 296.2], [443, 444], color='g', linestyle='--', linewidth=2)
    
#     for i,(x,y) in enumerate(points):
#         if i == (len(points) - 1):
#             xs = [x,points[0,0]]
#             ys = [y,points[0,1]]
#         else:
#             xs = [x,points[i+1,0]]
#             ys = [y,points[i+1,1]]
#         ax.plot(xs,ys,color='k',linestyle='--', linewidth=2)

    ax.plot(Xs,Zs,'r.',markersize=5)
    ax.plot(cart_Xs,[443.5]*len(cart_Xs),'c*',markersize=2)
    ax.set_xlim(xmin=286.5,xmax=296.2)
    ax.set_ylim(ymin=431,ymax=444)

    ax.invert_xaxis()
    #ax.axis('equal')
    ax.axis('off')
    if title: 
        ax.set_title(title, fontsize=22)   

In [ ]:
# # Visualization of cart and agent final pos reached on 1 run
# for m_name, m_id in zip(model_full_names,model_filenames):
#     b_k = saves[m_id][2]
#     print(b_k['parameters']['model_type'])
#     print(m_id)
#     #for i in range(nb_runs):
#     #b_k = saves[m_id][i]
#     #print(b_k['end_agent_x'][:10])
#     plot_agent_pos_exploration(1, b_k['end_agent_x'],
#                                    b_k['end_agent_z'],
#                                    b_k['end_cart_x'])
#     plt.show()

In [ ]:
# Compact Visu
def one_explo_maps(max_episodes=nb_episodes, run_nb=0, model_name='emmc3d_amb'):
    f, ax = plt.subplots(1,1,figsize=(5,5))
    b_k = saves[model_name][run_nb]
    print(b_k['parameters']['model_type'])
    plot_agent_pos_exploration(ax, b_k['end_agent_x'][:max_episodes],
                                b_k['end_agent_z'][:max_episodes],
                                b_k['end_cart_x'][:max_episodes], False)
    ax.plot(b_k['end_pickaxe_x'][:max_episodes],b_k['end_pickaxe_z'][:max_episodes],'g.',markersize=5)
    ax.plot(b_k['end_shovel_x'][:max_episodes],b_k['end_shovel_z'][:max_episodes],'b.',markersize=5)
    plt.tight_layout(pad=-1.0)
    
for i in range(nb_runs):
    one_explo_maps(max_episodes=40000, run_nb=i)
    plt.savefig("graphics/amb_exploration_map", bbox_inches='tight')
    plt.show()

In [ ]:
# Compact Visu
def compact_explo_maps(max_episodes=None, run_nb=0):
    f, (ax0, ax1, ax2, ax3) = plt.subplots(1,4,figsize=(20,5))
    ax = (ax0, ax1, ax2, ax3)
    for i, (m_name, m_id) in enumerate(zip(model_full_names,model_ids)):
        b_k = saves[m_id][run_nb]
        print(b_k['parameters']['model_type'])
        #print(m_id)
        #print(i)
        #for i in range(nb_runs):
        #b_k = saves[m_id][i]
        #print(b_k['end_agent_x'][:10])
        if max_episodes is None:
            plot_agent_pos_exploration(ax[i], b_k['end_agent_x'],
                                        b_k['end_agent_z'],
                                        b_k['end_cart_x'], titles[i])
            ax[i].plot(b_k['end_pickaxe_x'],b_k['end_pickaxe_z'],'g.',markersize=20)
            ax[i].plot(b_k['end_shovel_x'],b_k['end_shovel_z'],'b.',markersize=2)
        else:
            plot_agent_pos_exploration(ax[i], b_k['end_agent_x'][:max_episodes],
                                        b_k['end_agent_z'][:max_episodes],
                                        b_k['end_cart_x'][:max_episodes], titles[m_id])
            ax[i].plot(b_k['end_pickaxe_x'][:max_episodes],b_k['end_pickaxe_z'][:max_episodes],'g.',markersize=5)
            ax[i].plot(b_k['end_shovel_x'][:max_episodes],b_k['end_shovel_z'][:max_episodes],'b.',markersize=5)
            print(len(b_k['end_shovel_x'][:max_episodes]))
    #add pickaxe and shovel positions
        print(len(np.unique(b_k['end_cart_x'])))
    plt.tight_layout(pad=-1.0)

for i in range(nb_runs):
    compact_explo_maps(max_episodes=80000, run_nb=i)
    #plt.savefig("aa_newold_final2", bbox_inches='tight')
    #plt.show()

In [ ]:
# # GIF out of compact vizu
# step_size = 1000
# nb_steps = int(nb_episodes / step_size)
# filenames = []
# for i,max_episodes in enumerate(np.arange(step_size,nb_episodes+step_size,step_size)):
#     print(i)
#     compact_explo_maps(max_episodes)
#     f_name = "graphics/exploration_coverage_{}.png".format(i)
#     plt.savefig(f_name, bbox_inches='tight')
#     filenames.append(f_name)

# import imageio
# images = []
# for filename in filenames:
#     images.append(imageio.imread(filename))
# imageio.mimsave('graphics/exploration.gif', images, duration=0.3)

# GIF out of compact vizu
def explo_gif(run_nb=0):
    print("Making an exploration GIF for run nb {}".format(run_nb))
    step_size = 1000
    nb_steps = int(30000 / step_size)
    filenames = []
    for i,max_episodes in enumerate(np.arange(step_size,nb_episodes+step_size,step_size)):
        #print(i)
        compact_explo_maps(max_episodes,run_nb)
        f_name = "graphics/exploration_coverage_{}.png".format(i)
        plt.savefig(f_name, bbox_inches='tight')
        filenames.append(f_name)
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave('graphics/exploration{}.gif'.format(run_nb), images, duration=0.3)
for i in range(nb_runs):
    explo_gif(i)


![](graphics/exploration0.gif)
![](graphics/exploration1.gif)


In [ ]:
def get_nb_cells(x,y,nb_bins,rng):
    if y is not None: #2D case
        bin_numbers = sp.binned_statistic_2d(x, y, None, 'count',
                                                bins=nb_bins, range=rng).binnumber
    else: #1D case
        bin_numbers = sp.binned_statistic(x, None, 'count',
                                             bins=nb_bins, range=rng).binnumber
    return len(np.unique(bin_numbers))

# get env specific bounds
b = config.get_env_bounds('emmc_env')
nb_2d_cells = [10,30]
total_2d_cells = nb_2d_cells[0] * nb_2d_cells[1]
nb_c_cells = 200
nb_b_cells = 2**5

# Grid cell visualization
for m_id in model_ids:
    stat[m_id]['agent_cells'] = []
    stat[m_id]['pickaxe_cells'] = []
    stat[m_id]['shovel_cells'] = []
    stat[m_id]['cart_cells'] = []
    stat[m_id]['nb_cart_touched'] = []
    stat[m_id]['b_combinations'] = []
    stat[m_id]['first_blocks'] = []
    stat[m_id]['first_swing_left'] = []
    stat[m_id]['first_swing_right'] = []
    stat[m_id]['first_cart_touched'] = []
    stat[m_id]['eval_cart_touched'] = []
    stat[m_id]['chosen_modules'] = []
def mean_std(data_name):
    mean = round(np.mean(stat[m_id][data_name]),3)
    std = round(np.std(stat[m_id][data_name]),3)
    return mean,std

name_2d_spaces = ['agent', 'pickaxe', 'shovel']
for m_name, m_id in zip(model_full_names,model_ids):
    for i in range(nb_runs):
        b_k = saves[m_id][i]
        
        # grid cell for 2d outcome spaces (agent, pickaxe, shovel)
        for outc_space in name_2d_spaces:
            #print(outc_space+'_x')
            name = 'end_{}'.format(outc_space)
            bnd_x = b.get_bounds([outc_space+'_x'])
            bnd_z = b.get_bounds([outc_space+'_z'])
            bnds = bnd_x + bnd_z
            #print(bnds)
            nb_expl_cells = get_nb_cells(b_k[name+'_x'], b_k[name+'_z'],
                                         nb_2d_cells, bnds)
            stat[m_id][outc_space+'_cells'].append((nb_expl_cells/total_2d_cells)*100)
        
        #cart grid cell
        c_Xs = b_k['end_cart_x']
        nb_expl_cells = get_nb_cells(c_Xs, None,
                                     nb_c_cells, b.get_bounds(['cart_x']))
        stat[m_id]['cart_cells'].append((nb_expl_cells/nb_c_cells)*100)
        stat[m_id]['nb_cart_touched'].append(len(np.unique(b_k['end_cart_x'])))
        
        # first swing or touched car info
        try:
            first_cart_touched = next(x[0] for x in enumerate(c_Xs) if x[1] != 291.5)
        except StopIteration:
            first_cart_touched = 'never'
        # check for cart swing up
        try:
            first_swing_left = next(x[0] for x in enumerate(c_Xs) if x[1] > 296.2)
        except StopIteration:
            first_swing_left = 'never'
        try:
            first_swing_right = next(x[0] for x in enumerate(c_Xs) if x[1] < 286.8)
        except StopIteration:
            first_swing_right = 'never'
        
        #blocks
        b_vec = [b_k['end_block_0'], b_k['end_block_1'], b_k['end_block_2'],
                 b_k['end_block_3'], b_k['end_block_4']]
        b_vec = np.array(b_vec).reshape(-1,5)
        stat[m_id]['b_combinations'].append(len(np.unique(b_vec, axis=0)))
        
    
        
#         #blocks cell
#         first_blocks_found = []
#         for i in range(3):
#             try:
#                 first_found = nb_blocks.index(i)
#             except ValueError:
#                 first_found = 'never'
#             nb_found = nb_breads.count(float(i))
#             first_breads_found.append([first_found,nb_found])
#         #print( first_breads_found

#         final_bread_vec = np.zeros((len(bread_vec[0]),5))
#         #reconstruct original bread vectors
#         for bread_idx, v in enumerate(bread_vec):
#             for iter_idx in range(len(v)):
#                 final_bread_vec[iter_idx,bread_idx] = v[iter_idx]

#         b_explored_bins = len(np.unique(final_bread_vec,axis=0))
        
        

        
#         a_explored_bins, c_explored_bins, b_explored_bins, first_swing_left, first_swing_right, first_breads_found,\
#         first_cart_touched = get_grid_cell_exploration(b_k['end_agent_x'],
#                               b_k['end_agent_z'],
#                               b_k['end_cart_x'],
#                               b_k['end_bread_recovered'],
#                               [b_k['block_0'],b_k['block_1'],b_k['block_2']],
#                               c_bins=nb_c_cells,a_bins_x=nb_a_cells_x, a_bins_z=nb_a_cells_z, info=False)
#         stat[m_id]['agent_cells'].append((a_explored_bins/nb_a_cells)*100)
#         stat[m_id]['cart_cells'].append((c_explored_bins/nb_c_cells)*100)
#         stat[m_id]['b_combinations'].append((b_explored_bins/nb_b_cells)*100)
#         stat[m_id]['first_breads'].append(first_breads_found) 
        stat[m_id]['first_swing_left'].append(first_swing_left) 
        stat[m_id]['first_swing_right'].append(first_swing_right)
        stat[m_id]['first_cart_touched'].append(first_cart_touched)
        
print('--- EXPLORATION ---')
exploration_metrics = ['agent_cells', 'cart_cells', 'b_combinations',
                       'first_cart_touched', 'nb_cart_touched', 'first_swing_left', 'first_swing_right']

for m_id in model_ids:
    m_name = model_full_names[m_id]
    print("### " + m_name + " ####")
    
    for i in range(nb_runs):
        
        b_k = saves[m_id][i]
        a = np.array(b_k['choosen_modules'])
        unique, counts = np.unique(a, return_counts=True)
        #print dict(zip(unique, counts))
    
    for metric in exploration_metrics:
        values = stat[m_id][metric]
        if not 'first' in metric:
            mean,std = mean_std(metric)
            values = [round(v,1) for v in values]
            print(metric + ':' + '%s, MEAN: %s, STD: %s' % (values, mean, std))
        else:
            print(metric + ':' + '%s' % (values))

#     for i in range(6):
#         first_found = []
#         nb_found = []
#         for j in range(nb_runs):
#             first_found.append(stat[m_id]['first_breads'][j][i][0])
#             nb_found.append(stat[m_id]['first_breads'][j][i][1])
#         print('bread %s:' % i)
#         print("first found: %s" % first_found)
#         print("nb_found: %s" % nb_found)   


In [ ]:
# Grid cell exploration curves
step_size = 2000
nb_steps = int(nb_episodes / step_size)
print('ploting using {} steps'.format(nb_steps))
all_a_explo = np.zeros((nb_models,nb_runs,nb_steps+1))
all_p_explo = np.zeros((nb_models,nb_runs,nb_steps+1))
all_s_explo = np.zeros((nb_models,nb_runs,nb_steps+1))
all_c_explo = np.zeros((nb_models,nb_runs,nb_steps+1))
all_b_explo = np.zeros((nb_models,nb_runs,nb_steps+1))
name_2d_spaces = ['agent', 'pickaxe', 'shovel']
for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_ids)):
    print(m_name)
    for i in range(nb_runs):
        b_k = saves[m_id][i]
        a_explo = [0.]
        p_explo = [0.]
        s_explo = [0.]
        c_explo = [0.]
        b_explo = [0.]
        for max_episodes in np.arange(step_size,nb_episodes+step_size,step_size):
            
            for outc_space in name_2d_spaces:
                #print(outc_space+'_x')
                name = 'end_{}'.format(outc_space)
                bnd_x = b.get_bounds([outc_space+'_x'])
                bnd_z = b.get_bounds([outc_space+'_z'])
                bnds = bnd_x + bnd_z
                Xs = b_k[name+'_x'][:max_episodes]
                Zs = b_k[name+'_z'][:max_episodes]
                expl_cells = get_nb_cells(Xs, Zs, nb_2d_cells, bnds)
                if outc_space == 'agent':
                    a_expl_cells = expl_cells
                elif outc_space == 'pickaxe':
                    p_expl_cells = expl_cells
                elif outc_space == 'shovel':
                    s_expl_cells = expl_cells
        
            #cart grid cell
            c_Xs = b_k['end_cart_x'][:max_episodes]
            c_expl_cells = get_nb_cells(c_Xs, None,
                                         nb_c_cells, b.get_bounds(['cart_x']))
            
            
            a_explo.append((a_expl_cells/total_2d_cells)*100)
            s_explo.append((s_expl_cells/total_2d_cells)*100)
            p_explo.append((p_expl_cells/total_2d_cells)*100)
            
            c_explo.append((c_expl_cells/nb_c_cells)*100)
            
            #blocks
            b_vec = [b_k['end_block_0'][:max_episodes],
                     b_k['end_block_1'][:max_episodes],
                     b_k['end_block_2'][:max_episodes],
                     b_k['end_block_3'][:max_episodes],
                     b_k['end_block_4'][:max_episodes]]
            b_vec = np.array(b_vec).reshape(-1,5)
            b_explored_combs = len(np.unique(b_vec, axis=0))
            b_explo.append((b_explored_combs/nb_b_cells)*100)
        all_a_explo[m_nb,i,:] = a_explo
        all_p_explo[m_nb,i,:] = p_explo
        all_s_explo[m_nb,i,:] = s_explo
        all_c_explo[m_nb,i,:] = c_explo
        all_b_explo[m_nb,i,:] = b_explo
        
#print np.sum(all_a_explo,axis=1).shape
#print np.mean(np.sum(all_a_explo,axis=1),axis=0).shape
#break
colors = ['red','blue','green','magenta','black','cyan','orange']
mean_a_explo = np.mean(all_a_explo,axis=1)
mean_p_explo = np.mean(all_p_explo,axis=1)
mean_s_explo = np.mean(all_s_explo,axis=1)
mean_b_explo = np.mean(all_b_explo,axis=1)
mean_c_explo = np.mean(all_c_explo,axis=1)

std_a_explo = np.std(all_a_explo,axis=1)
std_p_explo = np.std(all_p_explo,axis=1)
std_s_explo = np.std(all_s_explo,axis=1)
std_b_explo = np.std(all_b_explo,axis=1)
std_c_explo = np.std(all_c_explo,axis=1)

In [ ]:
def plot_with_std(subplot_nb, ax,x,y,err,color,shade_color,label,
                  y_min=None,y_max=None, legend=False, title=None):
    #plt.rcParams.update({'font.size': 15})
    ax.locator_params(axis='x', nbins=5)
    ax.locator_params(axis='y', nbins=5)
    ax.plot(x,y, color=color, label=label,linewidth=1.5)
    ax.fill_between(x,y-err,y+err,color=shade_color,alpha=0.5)
    if legend:
        leg = ax.legend(loc='bottom right', fontsize=25)
        for legobj in leg.legendHandles:
            legobj.set_linewidth(4.0)
    ax.set_xlabel('iterations', fontsize=18)
    if subplot_nb == 0:
        ax.set_ylabel('Exploration %', fontsize=18)
    ax.set_xlim(xmin=min(x),xmax=max(x))
    if y_min is not None:
        ax.set_ylim(bottom=y_min,top=y_max)
    if title:
        ax.set_title(title, fontsize=22)

def plot_all_and_median(subplot_nb, ax,x,ys,color,label,
                         y_min=None,y_max=None, legend=False, title=None):
    ax.locator_params(axis='x', nbins=5)
    ax.locator_params(axis='y', nbins=5)
    #plot all
    added_legend = False
#     ys = [y.tolist() for y in ys]
#     ys.sort(key=lambda x: x[-1])
#     for k,y in enumerate(ys):
#         if added_legend:
#             label = None
#         else:
#             added_legend = True
#         if k == int(nb_runs/2):
#             linewidth = 4
#         else:
#             linewidth = 0.3
#         ax.plot(x,y, color=color, label=label,linewidth=linewidth)
    median = np.median(ys,axis=0)
    for k,y in enumerate(ys):
        if added_legend:
            label = None
        else:
            added_legend = True
        ax.plot(x,y, color=color, label=label,linewidth=0.3)
    ax.plot(x,median, color=color, label=label,linewidth=4)
    if legend:
        leg = ax.legend(loc='bottom right', fontsize=16)
        for legobj in leg.legendHandles:
            legobj.set_linewidth(4.0)
    ax.set_xlabel('iterations', fontsize=18)
    if subplot_nb == 0:
        ax.set_ylabel('Exploration %', fontsize=18)
    ax.set_xlim(xmin=min(x),xmax=max(x))
    if y_min is not None:
        ax.set_ylim(bottom=y_min,top=y_max)
    if title:
        ax.set_title(title, fontsize=22)
        

In [ ]:
metrics_mean_explo = [mean_a_explo, mean_p_explo, mean_s_explo, mean_b_explo, mean_c_explo] 
metrics_std_explo = [std_a_explo, std_p_explo, std_s_explo, std_b_explo, std_c_explo]
plot_steps = np.arange(0,nb_episodes+step_size,step_size)
#colors = ['red','blue','green','magenta','black','cyan','orange']
shades_colors = ['lightsalmon','skyblue','lightgreen','violet']
title_info = ["agent", "pickaxe", "shovel", "blocks", "cart"]
model_names = titles
metric = ["agent","pickaxe","shovel","blocks","cart"]
maxes=[85,85,85,100,85]

# # Averaged plots
# f, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(1,5,figsize=(20,4))
# ax = (ax0, ax1, ax2, ax3, ax4)
# print(model_filenames)
# for j, (models_mean_explo, models_std_explo) in enumerate(zip(metrics_mean_explo, metrics_std_explo)):
#     for i,(mean_explo, std_explo) in enumerate(zip(models_mean_explo, models_std_explo)):
#         y = mean_explo
#         err = std_explo
#         title = ''
#         plot_with_std(j, ax[j], plot_steps,y,err,colors[i],shades_colors[i],model_names[i],
#                       y_min=0,y_max=maxes[j], title=title+title_info[j], legend=True)
        
# #plt.savefig("graphics/grid_cell_exploration_mmc.pdf", bbox_inches='tight')
# #plt.savefig("grid_cell_exploration_samebt.png", bbox_inches='tight')
# plt.show()

#PLOT ALL + MEDIAN
all_explo = [all_a_explo, all_p_explo, all_s_explo, all_b_explo, all_c_explo]
f, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(1,5,figsize=(20,4))
ax = (ax0, ax1, ax2, ax3, ax4)
for j, models_all_explo in enumerate(all_explo):
    for i, (model_id, explo) in enumerate(zip(saves.keys(),models_all_explo)):
        ys = explo
        title = ''
        plot_all_and_median(j, ax[j], plot_steps,ys,model_colors[model_id],model_names[model_id],
                            y_min=0,y_max=maxes[j], title=title+title_info[j],
                            legend=True)
plt.savefig("graphics/grid_cell_exploration_mmc.png", bbox_inches='tight')


In [ ]:
def plot_with_std(x,y,err,color,shade_color,label,y_min,y_max):
    plt.plot(x,y, color=color, label=label,linewidth=1.5)
    plt.fill_between(x,y-err,y+err,color=shade_color,alpha=0.5)
    leg = plt.legend(loc='lower right')
    for legobj in leg.legendHandles:
        legobj.set_linewidth(4.0)
    plt.xlabel('episodes')
    plt.ylabel('exploration %')
    if y_min is not None:
        plt.ylim(ymin=y_min,ymax=y_max)
    

labels = ['RND','F-RGB','RMB','AMB']
shades_colors = ['lightsalmon','skyblue','lightgreen','violet']
print("agent exploration")
x = np.arange(0,nb_episodes+step_size,step_size)
for i in range(nb_models):
    print(std_a_explo[i,-1])
    y = mean_a_explo[i,:]
    err = std_a_explo[i,:]
    plot_with_std(x,y,err,colors[i],shades_colors[i],labels[i],0,100)
#plt.savefig("dist_agent_pos_expl.pdf", bbox_inches='tight')
plt.show()

print("shovel exploration")
x = np.arange(0,nb_episodes+step_size,step_size)
for i in range(nb_models):
    print(std_s_explo[i,-1])
    y = mean_s_explo[i,:]
    err = std_s_explo[i,:]
    plot_with_std(x,y,err,colors[i],shades_colors[i],labels[i],0,100)
#plt.savefig("dist_agent_pos_expl.pdf", bbox_inches='tight')
plt.show()

print("pickaxe exploration")
x = np.arange(0,nb_episodes+step_size,step_size)
for i in range(nb_models):
    print(std_p_explo[i,-1])
    y = mean_p_explo[i,:]
    err = std_p_explo[i,:]
    plot_with_std(x,y,err,colors[i],shades_colors[i],labels[i],0,100)
#plt.savefig("dist_agent_pos_expl.pdf", bbox_inches='tight')
plt.show()

print("cart")
for i in range(nb_models):
    print(std_c_explo[i,-1])
    y = mean_c_explo[i,:]
    err = std_c_explo[i,:]
    plot_with_std(x,y,err,colors[i],shades_colors[i],labels[i],0,100)
#plt.savefig("dist_cart_pos_expl.pdf", bbox_inches='tight')
plt.show()

print("blocks")
for i in range(nb_models):
    y = mean_b_explo[i,:]
    err = std_b_explo[i,:]
    plot_with_std(x,y,err,colors[i],shades_colors[i],labels[i],0,100)
#plt.savefig("dist_bread_pos_expl.pdf", bbox_inches='tight')
plt.show()
#plt.plot(mean_a_explo, color='red', label=m_name.replace('_',' '))
#plt.plot(mean_c_explo, color='blue', label=m_name.replace('_',' '))
#plt.plot(mean_b_explo, color='green', label=m_name.replace('_',' '))
        

In [ ]:
def plot_interests(ax,subplot_nb,interest_dict, legend=True, labels=None):
    ax.locator_params(axis='x', nbins=5)
    ax.locator_params(axis='y', nbins=5)
    colors = ['red','blue','green','magenta','black',
              'cyan','orange','powderblue','salmon','aqua','darkolivegreen',
             'orchid','indigo','saddlebrown','black', 'black']
    for i,(name,interests) in enumerate(sorted(interest_dict.items())):
        #print(name)
        if labels is not None:
            ax.plot(interests, color=colors[i], label=labels[i],linewidth=1.5)
        else:
            ax.plot(interests, color=colors[i], label=name.replace('_',' '),linewidth=1.5)
    if subplot_nb == 0:
        ax.set_ylabel('Intrinsic Rewards', fontsize=18)
        
    ax.set_xlabel('Iterations', fontsize=18)
    if legend:
        leg = ax.legend(loc='upper left')
        for legobj in leg.legendHandles:
            legobj.set_linewidth(4.0)

short_labels = ['agent','bread','cart','distr 1','distr 2','distr 3', 'distr 4']

# Plot interest curves:

for i in range(nb_runs):
    f, (ax0, ax1) = plt.subplots(1,2,figsize=(16,4))
    ax = (ax0, ax1)
    ints = saves['amb'][i]['interests']
    plot_interests(ax[0],0,ints,legend=True,labels=None)
#plt.savefig("aaa_newold_interests_samebt.png", bbox_inches='tight')




In [ ]:
# PLOT 3 SAMPLES OF AMB INTERESTS
f, (ax0, ax1, ax2) = plt.subplots(1,3,figsize=(24,4))
ax = (ax0, ax1, ax2)
for i,run_nb in enumerate([0,3,8]):
    ints = saves[model_filenames[2]][run_nb]['interests']
    plot_interests(ax[i],i,ints,legend=True,labels=None)
plt.savefig("graphics/interest_curves_amb_mmc.png", bbox_inches='tight')


In [ ]:
# Plot competence evolution for amb runs
f, (ax0, ax1) = plt.subplots(1,2,figsize=(20,4))
ax = (ax0, ax1)
m_names = ['pickaxe_end_pos']

for i in range(1):
    dists = {}
    for m_name in m_names:
        
        all_goals = saves[model_filenames[2]][1]['modules'][m_name].generated_goals
        all_outs = saves[model_filenames[2]][1]['modules'][m_name].observed_outcomes
        print(len(all_goals))
        print(len(all_outs))
        goals = np.array([all_goals[k-1] for k in range(5,len(all_goals),5)])
        outs = np.array([all_outs[k-1] for k in range(5,len(all_outs),5)])
        print(goals[-5:])
        print(outs[-5:])
        d = np.linalg.norm(goals - outs,axis=1)
        dists[m_name] = d
    #euc_d2 = np.linalg.norm(goals - outs,axis=1)
#     print(goals[0:3])
#     print(outs[0:3])
#     print(dists[0:3])
#     print(euc_d[0:3])
#     print(euc_d2[0:3])
#     print("h")
    plot_interests(ax[i],i,dists,legend=False,labels=None)

In [ ]:
# get interest curves names:
labels_2 = [s.replace('_',' ') for s in np.unique(saves[model_filenames[3]][0]['choosen_modules'])]

#labels = ['agent final position','bread final count','cart final position',
#               'fixed distractor 1 final position','fixed distractor 2 final position','moving distractor 1 final position', 'moving distractor 2 final position']
colors = ['red','blue','green','magenta','black','cyan','orange']

fig = pylab.figure()
figlegend = pylab.figure(figsize=(3.5,2))
ax = fig.add_subplot(111)
line_data = []
#for i in range(len(labels)):
#    line_data.append([range(10), pylab.randn(10), colors[i]])
print(len(labels_2))
if len(labels_2) == 3:
    lines = ax.plot(range(10),pylab.randn(10),colors[0],
                    range(10),pylab.randn(10),colors[1],
                    range(10),pylab.randn(10),colors[2])
elif len(labels_2) == 5:
    lines = ax.plot(range(10),pylab.randn(10),colors[0],
                    range(10),pylab.randn(10),colors[1],
                    range(10),pylab.randn(10),colors[2],
                    range(10),pylab.randn(10),colors[3],
                    range(10),pylab.randn(10),colors[4])
    
leg = figlegend.legend(lines, labels_2[:len(labels_2)], 'center')
for legobj in leg.legendHandles:
            legobj.set_linewidth(6.0)
        
figlegend.show()
figlegend.savefig('legend.png')

In [ ]:
# Time complexity analysis
for m_name, m_id in zip(model_full_names,model_filenames):
    print(m_id)
    b_k = saves[m_id][4]
    if 'runtimes' in b_k:
        rts = b_k['runtimes']
        #print(rts['produce'][-100:])
        plt.plot(rts['produce'],label='choose policy')
        plt.plot(rts['run'],label='run episode')
        plt.plot(rts['perceive'],label='add to memory')
        plt.legend()
        plt.ylabel('seconds')
        plt.xlabel('episodes')
    plt.savefig("time_complexity.png")
    plt.show()


In [ ]:
# PLOT percentage of modules that moved specific objects
def plot_explo_per_module(ax, b_k, obj_labels, initial_pos, nb_steps, spaces=False, legend=False, title=None):
    ax.locator_params(axis='x', nbins=5)
    ax.locator_params(axis='y', nbins=5)
    #first get list of idx where object moved for each module
    outcomes = []
    nb_outs = len(b_k[obj_labels[0]])
    nb_bt = nb_outs - len(b_k['choosen_modules'])
    #print(nb_bt)
    #print("adding {} bootstrap to module list".format(nb_bt))
    modules = ["random"]*nb_bt + b_k['choosen_modules']
    for i in range(len(b_k[obj_labels[0]])):
        outcomes.append([b_k[obj][i] for obj in obj_labels])
    #print(len(modules))
    #print(outcomes[100:150])
    #return
    
    
    dict_module_idx = {}
    for m_name in np.unique(modules):
        #first list -> when the module was choosen
        #second list -> when the module moved the object
        dict_module_idx[m_name] = [np.zeros(nb_outs), np.zeros(nb_outs)]
    for i, (outc, module_name) in enumerate(zip(outcomes, modules)):
        dict_module_idx[module_name][0][i] = 1
        if outc != initial_pos:
            dict_module_idx[module_name][1][i] = 1
    
    # now plot
    for m_name, info in dict_module_idx.items():
        #print(m_name)
        used = info[0]
        counts = info[1]
        d = []
        x = []
        for max_episodes in np.arange(step_size,nb_episodes+step_size,step_size):
            d.append(np.sum(counts[max_episodes-step_size:max_episodes]) / np.sum(used[max_episodes-step_size:max_episodes]))
            x.append(max_episodes)
        if spaces and m_name in spaces:
            if legend:
                label = m_name
            else:
                label = None
            ax.plot(x,d, label=label,linewidth=1.5, color=colors[spaces.index(m_name)])
    if legend: 
        ax.legend(fontsize=18)
    ax.set_xlabel('Iterations', fontsize=18)
    ax.set_ylabel('Prob. to move object', fontsize=18)
    if title:
        ax.set_title(title, fontsize=22)
        
    
   

In [ ]:
step_size = 2000
nb_steps = int(nb_episodes / step_size)
print('PICKAXE')        
for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
    if m_nb != 1:
        continue
    for i in range(nb_runs):
        print(i)
        f, ax = plt.subplots(1,1,figsize=(8,4))
        print(m_name)
        b_k = saves[m_id][i]
        plot_explo_per_module(ax, b_k, ['end_pickaxe_x', 'end_pickaxe_z'], [292.5, 436.5], nb_steps)
        plt.legend()
        #plt.savefig("oldnew_explo_mods_pickaxe{}{}.png".format(m_name,i), bbox_inches='tight')
        plt.show()
# print('SHOVEL')        
# for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
#     if m_nb == -1:
#         continue
#     for i in range(nb_runs):
#         print(i)
#         f, ax = plt.subplots(1,1,figsize=(8,4))
#         print(m_name)
#         b_k = saves[m_id][i]
#         plot_explo_per_module(ax, b_k, ['end_shovel_x', 'end_shovel_z'], [290.5, 436.5], nb_steps)
#         plt.legend()
#         plt.savefig("oldnew_explo_mods_shovel{}{}.png".format(m_name,i), bbox_inches='tight')
#         plt.show()

# print('BLOCKS')
# for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
#     if m_nb == -1:
#         continue
#     for i in range(nb_runs):
#         print(i)
#         f, ax = plt.subplots(1,1,figsize=(8,4))
#         print(m_name)
#         b_k = saves[m_id][i]
#         initial_v = [-1.0, -1.0, -1.0, -1.0, -1.0]
#         labels = ['end_block_0','end_block_1','end_block_2', 'end_block_3','end_block_4']
#         plot_explo_per_module(ax, b_k, labels, initial_v, nb_steps)
#         plt.legend()
#         plt.savefig("oldnew_explo_mods_blocks{}{}.png".format(m_name,i), bbox_inches='tight')
#         plt.show()
        

print('CART')
for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
    if m_nb != 1:
        continue
    for i in range(nb_runs):
        print(i)
        f, ax = plt.subplots(1,1,figsize=(8,4))
        print(m_name)
        b_k = saves[m_id][i]
        initial_v = [291.5]
        labels = ['end_cart_x']
        #plt.savefig("newnew_explo_mods_cart{}{}.png".format(m_name,i), bbox_inches='tight')
        plot_explo_per_module(ax, b_k, labels, initial_v, nb_steps)
        plt.legend()
        plt.show()

In [ ]:
# ALL IN ONE PLOT
step_size = 2000
nb_steps = int(nb_episodes / step_size)
f, (ax0, ax1) = plt.subplots(1,2,figsize=(14,4))
ax = (ax0, ax1)
print('PICKAXE')       
spaces = ['agent', 'pickaxe', 'blocks', 'cart' ,'random']
for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
    if m_nb != 1:
        continue
    for i in range(nb_runs):
        #print(i)
        #print(m_name)
        b_k = saves[m_id][i]
        plot_explo_per_module(ax[0], b_k, ['end_pickaxe_x', 'end_pickaxe_z'],
                              [292.5, 436.5], nb_steps, spaces=spaces, title='Pickaxe')

print('CART')
for m_nb,(m_name, m_id) in enumerate(zip(model_full_names,model_filenames)):
    if m_nb != 1:
        continue
    for i in range(nb_runs):
        #print(i)
        #print(m_name)
        b_k = saves[m_id][i]
        initial_v = [291.5]
        labels = ['end_cart_x']
        if i == nb_runs - 1:
            legend = True
        else:
            legend = False
        plot_explo_per_module(ax[1], b_k, labels, initial_v, nb_steps, spaces=spaces, legend=legend, title='Cart')
plt.savefig("graphics/transfer_mmc.png".format(m_name,i), bbox_inches='tight')
plt.show()